<a href="https://colab.research.google.com/github/Medynal/Pollution/blob/main/pollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/Medynal/Pollution.git

fatal: destination path 'Pollution' already exists and is not an empty directory.


In [31]:
!pip install gdown -U --no-cache-dir
from pathlib import Path
import gdown
gdrive_link = 'https://drive.google.com/drive/folders/1pMpViNUBVnC4xB4fTtYF9VHwp7c9Ms7b?usp=sharing'
gdown.download_folder(gdrive_link, output= 'pollution_dataset', quiet= False)
print ('download complete!' )

Retrieving folder contents


Processing file 1KFDJiZmhgbjSW7xGmWP7yzxoQnm-38a0 Ahmedabad_data.csv
Processing file 19vlL9i6x-vZU4ctApF4p5-MF-TLTMkeu Aizawl_data.csv
Processing file 1BhBvdHklAievoh0IJBtqRLXMq7SKNm_Y Amaravati_data.csv
Processing file 1v0us8qYW1CKJjN_7sBevoC0aw7bJsCqB Amritsar_data.csv
Processing file 1flxEMgV9JJAB7WzCKLKK4pBfB9D_aQ0l Bengaluru_data.csv
Processing file 1dZgKMW_hGX50h292KbH2aUObjypgX7O8 Bhopal_data.csv
Processing file 1PyFb5ZE3nJFdSYqmEuGcY1DsbVq2jihO Brajrajnagar_data.csv
Processing file 1byKiqT-hEN0IOXMfn2Wu3S65giQkGtK7 Chandigarh_data.csv
Processing file 18Cl3dgp_7TVbEJrhJ6woRjBv2QRQh0np Chennai_data.csv
Processing file 12l_Fd0Wv2YCSefvVXKrgBhiVud5ITEO5 Coimbatore_data.csv
Processing file 1fXTwPr782EJo3IJq8gb-nUYi0VBDEbfk Delhi_data.csv
Processing file 1OqbcppqTkahM7QRYDmE4iEZRyNQJx8vU Ernakulam_data.csv
Processing file 1erU1gOL6f-iNpAbtT92f-NQIpC4nK5Ed Gurugram_data.csv
Processing file 16iTfGdXEjcJtx8BrYwq90mulJXwvtGfA Guwahati_data.csv
Processing file 19duFvC3Pq3izBse2iF4ZLwfWrAh

Retrieving folder contents completed
Building directory structure
Building directory structure completed


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1KFDJiZmhgbjSW7xGmWP7yzxoQnm-38a0

but Gdown can't. Please check connections and permissions.

In [ ]:
# importing the necessary libraries for data analysis
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
dataframes = []
for filename in os.listdir('pollution_dataset'):
    if filename.endswith('.csv'):  # Check if the file is a CSV file
        file_path = os.path.join('pollution_dataset', filename)
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        dataframes.append(df)  # Add the DataFrame to the list

In [ ]:
pollution_df = pd.concat(dataframes, ignore_index=True)
pollution_df

In [ ]:
print ('shape', pollution_df.shape )

def unique_col(df):
  for column in df.columns:
    print (f'{column}: {df[column].nunique()}')

print(f'\nnumber of unigue value in each column')
unique_col(pollution_df)
print ('\n')
pollution_df.info()

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum() # Total missing values
    mis_val_percent = 100 * mis_val / len(df)  # Percentage of missing values
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})
    mis_val_table = mis_val_table.sort_values('% of Total Values', ascending=False)  # Sort the table by percentage of missing descending
    return mis_val_table

missing_values = missing_values_table(pollution_df)
display(missing_values.style.background_gradient(cmap='Blues'))

In [ ]:
pollution_df['Date']= pd.to_datetime(pollution_df['Date'], errors= 'raise',format= '%d/%m/%Y')
pollution_df['year']= pollution_df['Date'].dt.year
pollution_df['month']= pollution_df['Date'].dt.month
pollution_df['day']= pollution_df['Date'].dt.day
pollution_df['Month name']= pollution_df['Date'].dt.month_name()


In [ ]:
missing_values_per_city = pollution_df.groupby('City').apply(lambda x: x.isnull().sum())
missing_values_per_city

In [ ]:
plt.figure(figsize= (10,6))
sns.heatmap(missing_values_per_city)
plt.xlabel('Pollutant')
plt.show()

In [ ]:
pollution_df= pollution_df.sort_values(['City', 'Date'],ascending= [True, True])

In [ ]:
pollutants = ["PM2.5","PM10","NO","NO2","NOx","NH3","CO","SO2","O3",
              "Benzene","Toluene","Xylene", "AQI", "AQI_Bucket"]

pollution_df[pollutants] = pollution_df.groupby(['City', 'year'])[pollutants].ffill().bfill()
pollution_df.isna().sum()

In [ ]:

pollution_df.head(20)

In [ ]:
pollution_df.describe().T

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

pollutant_columns = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']

#Group by station and calculate mean pollutant concentrations
mean_pollutant_by_city =pollution_df.groupby('City')[pollutant_columns].mean()

#Find the top 5 stations for each pollutant
top_city = {}
for pollutant in pollutant_columns:
    top_city[pollutant] = mean_pollutant_by_city[pollutant].sort_values(ascending=False).head(5)

# Step 3: Plotting
fig, axes = plt.subplots(len(pollutant_columns), 1, figsize=(10, 20))

for i, pollutant in enumerate(pollutant_columns):
    axes[i].barh(top_city[pollutant].index, top_city[pollutant].values, color='red')
    axes[i].set_title(f'Top 5 Cities by {pollutant}')
    axes[i].set_xlabel(f'{pollutant}')
    axes[i].invert_yaxis()  # Highest values on top

plt.tight_layout()
plt.show()


In [ ]:
pollutants = ['PM2.5','PM10','NO2','NOx','NH3','CO', 'SO2','O3', 'Benzene', 'Toluene', 'Xylene']
cities = pollution_df['City'].unique()

for city in cities:
    city_data = pollution_df[pollution_df['City'] == city]
    plt.figure(figsize=(10, 6))
    for pollutant in pollutants:
        plt.bar(pollutant, city_data[pollutant].mean(), label=pollutant)
    plt.title(f'Average Pollutant Levels at Station {city}')
    plt.xlabel('Pollutant')
    plt.ylabel('Average Concentration')
    plt.legend()
    plt.show()

In [ ]:
 city_stats = pollution_df.groupby('City')[pollutants].mean().reset_index()

import plotly.express as px

# Melt the DataFrame to create a 'Pollutant' column
city_stats_melted = city_stats.melt(id_vars='City',
                               value_vars=pollutants,
                               var_name='Pollutant',
                               value_name='Average Concentration')

# Create the stacked bar chart
fig_cities= px.bar(city_stats_melted,
                     x='City',
                     y='Average Concentration',
                     color='Pollutant', # Use 'Pollutant' for color differentiation
                     title='Average Pollution Levels by City (Stacked)',
                     labels={'City': 'Cities', 'Average Concentration': 'Average Concentration'},
                     barmode='stack')

fig_cities.show()